In [ ]:
#Note to run all cells from top to bottom(i.e first to last)
#The below cell is used to convert the dataset into readable data for training and saves the X and Y values as pickle files.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "W:/Datasets/"
IMG_SIZE = 70
CATEGORIES = ["no_mask", "mask"]
training_data = []

def create_training_data():
    for category in CATEGORIES: 
        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):  
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
            training_data.append([new_array, class_num])  
        
create_training_data()

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

print('Done')

In [ ]:
#The below cell is used for training the model.
#First it reads both the training images and their labels
#Then it creates the CNN using two convolutional layers and 2 dense layers.
#It will split the dataset for a 0.3 validation split and 
#Save the model as CNN7.model

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pickle

NAME = "Newly and Fresh"

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y)

model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  

model.add(Dense(64))
model.add(Activation('relu'))
    
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=2, validation_split=0.3, callbacks=[tensorboard])

model.save('CNN7.model')

In [ ]:
#This block of code loads the model for predictions in the next block

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

model = tf.keras.models.load_model("CNN7.model")
print('Done')

In [ ]:
#This block of code is for predicting images using the model.
#It will print the image in gray scale as well as the according prediction
#To change the file you are predicting just change the file path

In [ ]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

CATEGORIES = ["no_mask", "mask"]


def prepare(filepath):
    IMG_SIZE = 70 
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    plt.imshow(new_array, cmap='gray')  
    plt.show()  
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

prediction = model.predict([prepare('W:/Datasets/mask.jpg')]) #Change file path for the image to be predicted
print(CATEGORIES[int(prediction[0][0])])

In [ ]:
#This block of code is used to iterate over the test dataset in order to test the accuracy of the model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "W:/Datasets/"
IMG_SIZE = 70
CATEGORIES = ["no_mask_test", "mask_test"]
CATEGORIES1 = ["no_mask", "mask"]


def check_accuracy():
    noMaskSize = 0
    maskSize = 0
    numCor = 0
    numInc = 0
    for category in CATEGORIES:  
        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):  
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
            prediction = model.predict([new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)])
            if (category == "mask_test"):
                maskSize = maskSize + 1
                numCor = numCor + int(prediction[0][0])
            if (category == "no_mask_test"):
                noMaskSize = noMaskSize + 1
                numInc = numInc + int(prediction[0][0])
    numTotal = noMaskSize + maskSize
    accuracy = (numCor + (noMaskSize - numInc))/numTotal
    print(accuracy)
    
check_accuracy()

